In [3]:
#!pip install tensorflow
#!pip install scikit-learn

   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
    --------------------------------------- 0.2/9.3 MB 4.1 MB/s eta 0:00:03
   ------------ --------------------------- 3.0/9.3 MB 31.7 MB/s eta 0:00:01
   --------------------------- ------------ 6.4/9.3 MB 40.7 MB/s eta 0:00:01
   ---------------------------------------  9.3/9.3 MB 42.2 MB/s eta 0:00:01
   ---------------------------------------- 9.3/9.3 MB 39.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/301.2 kB ? eta -:--:--
   --------------------------------------- 301.2/301.2 kB 18.2 MB/s eta 0:00:00


In [1]:
import numpy
import sklearn
import tensorflow
import pandas as pd

In [34]:
# Load the dataset
data = pd.read_csv('C:/Users/laura/Desktop/AI/styles.csv', on_bad_lines='skip')
data.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [35]:
# Clean the data (remove duplicates, handle missing values, etc.)
data = data.drop_duplicates()
data = data.dropna()

In [36]:
# Transform categorical variables into numerical representations
data['gender'] = pd.Categorical(data['gender']).codes
data['masterCategory'] = pd.Categorical(data['masterCategory']).codes
data['subCategory'] = pd.Categorical(data['subCategory']).codes
data['articleType'] = pd.Categorical(data['articleType']).codes
data['baseColour'] = pd.Categorical(data['baseColour']).codes
data['season'] = pd.Categorical(data['season']).codes
data['usage'] = pd.Categorical(data['usage']).codes

data

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,2,1,38,103,25,0,2011.0,0,Turtle Check Men Navy Blue Shirt
1,39386,2,1,6,56,2,2,2012.0,0,Peter England Men Party Blue Jeans
2,59263,4,0,42,139,37,3,2016.0,0,Titan Women Silver Watch
3,21379,2,1,6,127,1,0,2011.0,0,Manchester United Men Solid Black Track Pants
4,53759,2,1,38,133,13,2,2012.0,0,Puma Men Grey T-shirt
...,...,...,...,...,...,...,...,...,...,...
44419,17036,2,2,30,19,44,2,2013.0,0,Gas Men Caddy Casual Shoe
44420,6461,2,2,11,39,33,2,2011.0,0,Lotto Men's Soccer Track Flip Flop
44421,18842,2,1,38,133,2,0,2011.0,0,Puma Men Graphic Stellar Blue Tshirt
44422,46694,4,5,12,91,2,1,2017.0,0,Rasasi Women Blue Lady Perfume


In [37]:
from sklearn.metrics.pairwise import cosine_similarity
# Assuming we have a user-item matrix with clothing features
item_features = data[["gender", "masterCategory", "subCategory", "articleType", "baseColour", "season", "year", "usage"]]

In [38]:
# Calculate cosine similarity between item features
similarity_matrix = cosine_similarity(item_features)
similarity_matrix

array([[1.        , 0.99953563, 0.99982317, ..., 0.99982433, 0.99983015,
        0.99983624],
       [0.99953563, 1.        , 0.99885046, ..., 0.99914361, 0.99984409,
        0.99889737],
       [0.99982317, 0.99885046, 1.        , ..., 0.99984267, 0.99945453,
        0.9999956 ],
       ...,
       [0.99982433, 0.99914361, 0.99984267, ..., 1.        , 0.99969462,
        0.99988964],
       [0.99983015, 0.99984409, 0.99945453, ..., 0.99969462, 1.        ,
        0.99950149],
       [0.99983624, 0.99889737, 0.9999956 , ..., 0.99988964, 0.99950149,
        1.        ]])

In [39]:
similarity_matrix.shape

(44077, 44077)

In [40]:
# Create a similarity score feature for each item
item_similarity_scores = similarity_matrix.mean(axis=1)


In [62]:
def recommend_similar_items(item_name, top_n=5):
    # Check if item_name is in the dataset
    if item_name not in data['productDisplayName'].values:
        print(f"Item '{item_name}' not found in the dataset.")
        return
    
    # Find the index of the item in the dataset
    item_index = data[data['productDisplayName'] == item_name].index[0]
    
    # Get the similarity scores for the given item
    item_similarities = similarity_matrix[item_index]
    
    # Get indices of top similar items
    similar_indices = item_similarities.argsort()[::-1][1:top_n+1]  # Exclude itself
    
    # Get the names of top similar items
    similar_items = data.iloc[similar_indices]['productDisplayName'].values
    
    return similar_items


In [63]:
# Example usage:
user_input_item = input("Which item do you want similar recommendations for?")
recommended_items = recommend_similar_items(user_input_item)
print(f"Recommended items similar to '{user_input_item}':")
for item in recommended_items:
    print(item)

Recommended items similar to 'Turtle Check Men Navy Blue Shirt':
U.S. Polo Assn. Men Check Navy Blue Shirt
United Colors of Benetton Men Check Navy Blue Shirt
Wrangler Men Urban Navy Blue Shirt
Wrangler Men Easter Navy Blue Shirt
Probase Men Wide Stripes Navy Blue Shirts
